In [1]:
!pip install transformers

In [2]:
import torch
import transformers as ppb
import pandas as pd
import numpy as np

In [11]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'cse6242project-417410'
!gcloud config set project {project_id}

Are you sure you wish to set property [core/project] to cse6242project-417410?

Do you want to continue (Y/n)?  Y

Updated property [core/project].


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!curl https://sdk.cloud.google.com | bash

In [ ]:
bucket_name = 'team054'
# Download the file from the given Google Cloud Storage bucket.
!gsutil cp gs://team054/books_enriched.csv books_enriched.csv

You are attempting to access protected data with no configured
credentials. Please visit https://cloud.google.com/console#/project
and sign up for an account, and then run the "gcloud auth login"
command to configure gsutil to use these credentials.


In [4]:
import pandas as pd
from ast import literal_eval
#books_df = pd.read_csv('/content/books_enriched.csv')#---if uploading through the bucket way,then uncomment this

books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/books_enriched.csv')

#books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/combined_processed_files.txt')  --- Terry's file

books_df = books_df[:1000]

In [5]:
books_df['short_description'] = books_df.description.str.slice(0,512)

In [6]:


df = books_df.short_description

In [7]:
books_df2 = pd.read_table('/content/drive/MyDrive/CSE6242_project/gutenberg_books.txt', index_col=[0])

In [8]:
books_df2['short_description'] = books_df.description

In [9]:
# add all rows of books_df2['short_description'] to df

df = pd.concat([df, books_df2['short_description']])

In [12]:
tokenized = df.apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

In [13]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

# Embed Text

In [14]:
#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#PYTORCH_NO_CUDA_MEMORY_CACHING=1
##set CUDA_VISIBLE_DEVICES=""

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)
model = model.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [15]:
embeddings = last_hidden_states[0].cpu().data[:,0,:].numpy()

In [16]:
df.shape, embeddings.shape

((42723,), (42723, 768))

# Save Embeddings

In [22]:
embeddings_pd_df = pd.DataFrame({'book_id': range(df.shape[0])})
embeddings_pd_df['title'] = books_df['title']
embeddings_pd_df['embedding'] = embeddings.tolist()

In [23]:

embeddings_pd_df['embedding'] = embeddings_pd_df['embedding'].apply(lambda x: np.array(x))


In [ ]:
embeddings_pd_json= embeddings_pd_df.to_json()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
df.to_json('/content/drive/MyDrive/CSE6242_project/embeddings_pd_json.json')

In [ ]:
#!gsutil cp embeddings_pd_txt gs://team054

In [ ]:
#import datalab.storage as gcs
#gcs.Bucket('team054').item('to/embeddings_pd_json.json').write_to(simple_dataframe.to_json(),'text/json')

In [ ]:
#from google.cloud import storage

#def write_to_cloud(buffer):
 #   client = storage.Client()
  #  bucket = client.get_bucket('team054')
  #  blob = bucket.blob('gs://team054/embeddings_pd_json.json')
 #   blob.upload_from_file(buffer)

In [ ]:
#from google.cloud import storage

#def create_newfolder(bucket_name, destination_folder_name):
 #   storage_client = storage.Client()
 #   bucket = storage_client.get_bucket('team054')
  #  blob = bucket.blob('output')

  #  blob.upload_from_file('')

   # print('Created {} .'.format(destination_folder_name))

In [24]:
embeddings_pd_df.head()

,book_id,title,embedding
0,0,"The Hunger Games (The Hunger Games, #1)","[0.1194697842001915, 0.057542070746421814, 0.3..."
1,1,Harry Potter and the Sorcerer's Stone (Harry P...,"[-0.374781996011734, -0.272121787071228, 0.131..."
2,2,"Twilight (Twilight, #1)","[0.12502777576446533, -0.7245572805404663, -0...."
3,3,To Kill a Mockingbird,"[-0.43787628412246704, -0.29153966903686523, -..."
4,4,The Great Gatsby,"[-0.40598979592323303, -0.47040653228759766, -..."


# Train KNN

In [35]:
# cell edited by Tyler (3/24/2024)

from sklearn.neighbors import NearestNeighbors

# fit knn model
embedding_vals = np.array(embeddings_pd_df['embedding'].values.tolist())
k = 5
nn_model = NearestNeighbors(n_neighbors=k, algorithm="ball_tree").fit(embedding_vals)

In [36]:
# save the trained model to mydrive as pickle file
import cloudpickle

with open('/content/drive/MyDrive/CSE6242_project/knn_model.pkl', 'wb') as f:
  cloudpickle.dump(nn_model, f)